# Table Sigmoid Implementation

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from NeuralNetwork.tables import sigmoid_table, log_table, get_cross_entropy_tables

In [2]:
random_state = 124
np.random.seed(random_state)

EXTRA_BITS = 4
DECIMAL_BITS = 12
K_SHIFT = EXTRA_BITS + DECIMAL_BITS
PRECISION = "int32"
DOUBLE_PRECISION = "int32"

# Create sigmoid table
sigma_dec, sigma_int = sigmoid_table(EXTRA_BITS, DECIMAL_BITS, PRECISION)

In [3]:
table_size = sigma_int.shape[0]
unique_entries = (np.unique(sigma_int)).shape[0]
print("Table size = {}, Unique entries = {}".format(table_size, unique_entries))
print("Size if reduction can be made = {:.2f}%".format(100*(1-unique_entries/table_size)))

Table size = 65536, Unique entries = 2048
Size if reduction can be made = 96.88%


### Can we make the table smaller?

In [4]:
df = pd.DataFrame(np.arange(sigma_int.shape[0]), columns=["input"])
df["output"] = pd.DataFrame(sigma_int.reshape(-1,1))
df

,input,output
0,0,2047
1,1,2047
2,2,2047
3,3,2048
4,4,2048
...,...,...
65531,65531,4094
65532,65532,4094
65533,65533,4094
65534,65534,4094


In [5]:
data = df.groupby("output").indices
keys = data.keys()

for idx, k in enumerate(keys):
    if(idx==0):
        print('if (in >= {} && in <= {}) out = {};'.format(data[k].min(), data[k].max(), k))
    else:
        print('else if (in >= {} && in <= {}) out = {};'.format(data[k].min(), data[k].max(), k))

if (in >= 0 && in <= 2) out = 2047;
else if (in >= 3 && in <= 6) out = 2048;
else if (in >= 7 && in <= 10) out = 2049;
else if (in >= 11 && in <= 14) out = 2050;
else if (in >= 15 && in <= 18) out = 2051;
else if (in >= 19 && in <= 22) out = 2052;
else if (in >= 23 && in <= 26) out = 2053;
else if (in >= 27 && in <= 30) out = 2054;
else if (in >= 31 && in <= 34) out = 2055;
else if (in >= 35 && in <= 38) out = 2056;
else if (in >= 39 && in <= 42) out = 2057;
else if (in >= 43 && in <= 46) out = 2058;
else if (in >= 47 && in <= 50) out = 2059;
else if (in >= 51 && in <= 54) out = 2060;
else if (in >= 55 && in <= 58) out = 2061;
else if (in >= 59 && in <= 62) out = 2062;
else if (in >= 63 && in <= 66) out = 2063;
else if (in >= 67 && in <= 70) out = 2064;
else if (in >= 71 && in <= 74) out = 2065;
else if (in >= 75 && in <= 78) out = 2066;
else if (in >= 79 && in <= 82) out = 2067;
else if (in >= 83 && in <= 86) out = 2068;
else if (in >= 87 && in <= 90) out = 2069;
else if (in >= 91 && 

In [6]:
original_stdout = sys.stdout # Save a reference to the original standard output

with open('sigma_int.sv', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    
    print("module sigma_int(input logic [{}:0] in,".format(K_SHIFT-1))
    print("\t output logic [{}:0] out);".format(DECIMAL_BITS-1))
    print("\t \t always_comb")
   
    for idx, k in enumerate(keys):
        if(idx==0):
            print('\t \t \t if (in >= {} && in <= {}) out = {};'.format(data[k].min(), data[k].max(), k))
        else:
            print('\t \t \t else if (in >= {} && in <= {}) out = {};'.format(data[k].min(), data[k].max(), k))
    print("\t \t \telse out=4095;")
    print("\t endmodule")
    
    sys.stdout = original_stdout # Reset the standard output to its original value